<p align="center">
<img src="../assets/images/dtlogo.png" alt="Duckietown Logo" width="50%">
</p>

# Sending LED Messages with ROS

Now we can use the `LEDPattern` message that we defined in the first notebook to change the LEDs on the Duckiebot using ROS tools.

In this notebook, we'll use the `dt-gui-tools` interface to send ROS messages directly to the Duckiebot via the command line.

<p align="center">
<img src="../assets/images/lit_duckiebot.jpg" alt="Duckietown Logo" width="50%">
</p>

---

## Step 1: Tracing the Message Route

Who will actually read the LED messages when we send them? 

The Duckiebot is running an *led_emitter* ROS node which provides the [ROS service](http://wiki.ros.org/Services) *SetCustomLEDPattern*.  

If we call this service and provide an `LEDPattern` message, the emitter will construct and publish messages for the *led_driver_node*, which updates the individual LEDs.  The following graph illustrates this relationship

<p align="center">
<img src="../assets/images/rqt.png" alt="Duckietown LED Nodes" width="100%">
</p>

The *led_emitter* node is passing `LEDPattern` messsages to the *led_driver* node, which is also communicating with a few diagnostics topics.

**Note:** You can view the full ROS graph with all active nodes and messages on the Duckiebot dashboard under the *Architecture* tab.

## Step 2: Talking to the LED Driver

Before looking at the *SetCustomLEDPattern* service, let's explore the system at a lower level by passing our own messages directly to the *led_driver* node.

To update the LEDs using the command line, first run 

    dts start_gui_tools <your_duckiebot>

to obtain a terminal (container actually) that is connected to the Duckiebot ROS network. In this terminal you can perform all ROS commands and have access to all ROS messages streaming on your Duckiebot.

In the same terminal (now connected to the Duckiebot container), run

    rostopic list

to see a list of the published ROS topics on the Duckiebot.

You should see `/<your_duckiebot>/led_emitter_node/led_pattern` included in this list.  The *led_driver* node running on the Duckiebot is subscribed to this topic and will set the LEDs to their new values when the emitter publishes an `LEDPattern` message.

To confirm this, run

    rostopic info /your_duckiebot/led_emitter_node/led_pattern

you should see that the topic handles `Type: duckietown_msgs/LEDPattern`, is published by `led_emitter_node`, and subscribed to by `led_driver_node`.

The *led_driver* node is looking at the **rgb_vals** in this message to update each LED.

Publish your own message that will trigger a change to the LED colors with `rostopic pub` by running

    rostopic pub /<your_duckiebot>/led_emitter_node/led_pattern duckietown_msgs/LEDPattern "{rgb_vals: [{r: 1.0, g: 0.0, b: 0.0, a: 1.0}, {r: 1.0, g: 0.0, b: 0.0, a: 1.0}, {r: 0.0, g: 1.0, b: 0.0, a: 1.0}, {r: 0.0, g: 1.0, b: 0.0, a: 1.0}, {r: 1.0, g: 0.0, b: 0.0, a: 1.0}]}"

You should see the left LEDs are now red, while the right LEDs are green.  

<p align="center">
<img src="../assets/images/green_red.jpg" alt="Duckietown LEDs Green and Red" width="50%">
</p>

Try playing with a few different color combinations.

## Step 3: What about the LED Emitter?

You'll notice that you are missing some features by communicating directly with the *led_driver* node, because you are setting the state of individual LEDs.  For example, you won't be able to set a frequency for the LEDs to blink at - more complex behavior like this is handled by the `led_emitter`.

We need the *led_emitter* node to provide the logic required to create LED patterns and translate these behaviors into simple messages for the *led_driver* node.

This is where the *SetCustomLEDPattern* service we described earlier comes in.  We can tell the *led_emitter* to handle custom patterns with features such as frequency and intensity by calling the service and providing an `LEDPattern` message.

The *led_emitter* will be covered in more depth in later Learning Experiences.  For now, we will stay at the driver level and learn how to publish messages for the LED driver from a custom ROS node.  This will give you the foundation to use LEDs at any level in your projects.

## Next Up: Adding LED Patterns to an Agent

Continue on to the [next notebook](./03_Adding-Agent-LED-Behaviors.ipynb) to integrate `LEDPattern` messages into your Duckiebot agent.